In [50]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2019-04-22 16:56:46--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222, 2001:67c:1560:8003::8004, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-22 16:56:46 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
··········


In [51]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [52]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import cv2
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.datasets import load_files
import matplotlib.pyplot as plt

In [0]:
def load_data(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    return files

In [0]:
def ext_img(img):
    imag = cv2.imread(img)
    imag = cv2.cvtColor(imag,cv2.COLOR_BGR2GRAY)
    imag = imag/255
    imag = np.expand_dims(imag,axis=2)
    return imag

In [0]:
def tensor_4d(fil):
    lis = [ext_img(im) for im in tqdm(fil)]
    return np.stack(lis,axis=0)

In [0]:
fold_path = 'drive/datasets/shapes'

In [0]:
img_files = load_data(fold_path)

In [59]:
imgs = tensor_4d(img_files)

100%|██████████| 300/300 [00:00<00:00, 475.57it/s]


In [60]:
imgs.shape

(300, 28, 28, 1)

In [0]:
class Encoder(tf.keras.layers.Layer):
    
    def __init__(self):
        super(Encoder,self).__init__()
        self.conv8 = tf.keras.layers.Conv2D(8,3,activation=tf.nn.relu,strides=(1,1),padding='SAME')
        self.conv16 = tf.keras.layers.Conv2D(16,3,activation=tf.nn.relu,strides=(1,1),padding='SAME')
        self.conv32 = tf.keras.layers.Conv2D(32,3,activation=tf.nn.relu,strides=(1,1),padding='SAME')
        self.max_pool_1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))
        self.max_pool_2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))
        
    def call(self,inp):
        x = self.conv32(inp)
        x = self.max_pool_1(x)
        x = self.conv16(x)
        x = self.max_pool_2(x)
        x = self.conv8(x)
        return x
        
        
        

In [0]:
class Decoder(tf.keras.layers.Layer):
    
    def __init__(self):
        super(Decoder,self).__init__()
        self.convtr8 = tf.keras.layers.Conv2DTranspose(8,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.convtr16 = tf.keras.layers.Conv2DTranspose(16,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.convtr32 = tf.keras.layers.Conv2DTranspose(32,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.convtr1 = tf.keras.layers.Conv2DTranspose(1,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.upsample_1 = tf.keras.layers.UpSampling2D(size=(2,2))
        self.upsample_2 = tf.keras.layers.UpSampling2D(size=(2,2))
        
    def call(self,inp):
        x = self.convtr8(inp)
        x = self.upsample_1(x)
        x = self.convtr16(x)
        x = self.upsample_2(x)
        x = self.convtr32(x)
        x = self.convtr1(x)
        return x
        
        

In [0]:
class Autoencoder(tf.keras.Model):
    
    def __init__(self):
        super(Autoencoder,self).__init__()
        self.encode = Encoder()
        self.decode = Decoder()
        
    def call(self,inputs):
        encoded = self.encode(inputs)
        decoded = self.decode(encoded)
        return decoded

In [0]:
ae = Autoencoder()

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

In [73]:
(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255


11493376/11490434 [==============================] - 0s 0us/step


In [0]:
loss_metric = tf.keras.metrics.Mean()


In [84]:
for epoch in range(3):
    print('Start of epoch %d' % (epoch,))

  # Iterate over the batches of the dataset.
    for start,end in zip(range(0,len(imgs),10),range(10,len(imgs),10)):
        xtr = imgs[start:end]
        xtr = xtr.astype('float32')
        with tf.GradientTape() as tape:
            reconstructed = ae(xtr)
            # Compute reconstruction loss
            loss = mse_loss_fn(xtr, reconstructed)
            loss += sum(ae.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, ae.trainable_variables)
        #print(grads)
        optimizer.apply_gradients(zip(grads, ae.trainable_variables))

        loss_metric(loss)

        if start % 100 == 0:
            print('step %s: mean loss = %s' % (start, loss_metric.result()))

Start of epoch 0
step 0: mean loss = tf.Tensor(0.105098695, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.100677535, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.096619174, shape=(), dtype=float32)
Start of epoch 1
step 0: mean loss = tf.Tensor(0.09331678, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.08998443, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.08687782, shape=(), dtype=float32)
Start of epoch 2
step 0: mean loss = tf.Tensor(0.08431542, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.081707776, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.07924685, shape=(), dtype=float32)
